In [24]:
import matplotlib as mpl
import matplotlib.pyplot as plt

# Notebook içine gömülen görsellerin "ağırlığını" düşürür
mpl.rcParams["figure.dpi"] = 80
mpl.rcParams["savefig.dpi"] = 80
mpl.rcParams["figure.figsize"] = (10, 6)

# Çok büyük figürleri otomatik sınırlamak için (opsiyonel)
MAX_FIGSIZE = (12, 12)


**0.1 Kütüphaneler**

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from skimage.io import imread
from skimage.color import rgb2gray


**0.2 Veri Yolu**

In [26]:
DATA_PATH = "/content/drive/MyDrive/ISIC_data"


In [27]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**1 - RGB → Grayscale Dönüşümü**

**RGB → Gray öncesi kıl bastırma fonksiyonu**

In [28]:
import cv2
import numpy as np

def hair_removal_rgb(img_rgb):
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (17, 17))
    blackhat = cv2.morphologyEx(img_gray, cv2.MORPH_BLACKHAT, kernel)

    _, mask = cv2.threshold(blackhat, 10, 255, cv2.THRESH_BINARY)
    inpainted = cv2.inpaint(img_gray, mask, 1, cv2.INPAINT_TELEA)

    return inpainted


**1) RGB → Grayscale + Artefakt Bastırma (9 örnek)**

In [29]:
import matplotlib.pyplot as plt
from skimage.io import imread
import random

sample_paths = random.sample(image_paths, 9)

plt.figure(figsize=(12, 18))

for i, p in enumerate(sample_paths):
    rgb = imread(p)
    gray_clean = hair_removal_rgb(rgb)

    plt.subplot(9, 2, 2*i+1)
    plt.imshow(rgb)
    plt.title("RGB (Orijinal)")
    plt.axis("off")

    plt.subplot(9, 2, 2*i+2)
    plt.imshow(gray_clean, cmap="gray")
    plt.title("Grayscale (Artefakt Bastırılmış)")
    plt.axis("off")

plt.tight_layout()
plt.show()


**2 - Pre-Processing**

**2.1 Crop**


**Crop fonksiyonu**

In [30]:
def center_crop(img, crop_ratio=0.1):
    h, w = img.shape[:2]
    dh = int(h * crop_ratio)
    dw = int(w * crop_ratio)
    return img[dh:h-dh, dw:w-dw]


**2.1 Crop – 9 örnek görselleştirme**

In [31]:
sample_paths = random.sample(image_paths, 9)

plt.figure(figsize=(12, 18))

for i, p in enumerate(sample_paths):
    rgb = imread(p)
    gray_clean = hair_removal_rgb(rgb)
    cropped = center_crop(gray_clean, crop_ratio=0.1)

    plt.subplot(9, 2, 2*i+1)
    plt.imshow(gray_clean, cmap="gray")
    plt.title("Grayscale (Crop Öncesi)")
    plt.axis("off")

    plt.subplot(9, 2, 2*i+2)
    plt.imshow(cropped, cmap="gray")
    plt.title("Crop Sonrası")
    plt.axis("off")

plt.tight_layout()
plt.show()


**2.2 Kontrast İyileştirme**



**Histogram Equalization uygulaması**

In [32]:
from skimage import exposure

def contrast_enhancement(img_gray):
    img_eq = exposure.equalize_hist(img_gray)
    return img_eq


**9 örnek için görselleştirme**

In [33]:
sample_paths = random.sample(image_paths, 9)

plt.figure(figsize=(12, 18))

for i, p in enumerate(sample_paths):
    rgb = imread(p)
    gray_clean = hair_removal_rgb(rgb)
    cropped = center_crop(gray_clean, crop_ratio=0.1)
    enhanced = contrast_enhancement(cropped)

    plt.subplot(9, 2, 2*i+1)
    plt.imshow(cropped, cmap="gray")
    plt.title("Crop Sonrası")
    plt.axis("off")

    plt.subplot(9, 2, 2*i+2)
    plt.imshow(enhanced, cmap="gray")
    plt.title("Histogram Equalization")
    plt.axis("off")

plt.tight_layout()
plt.show()


**2.3 Gürültü Azaltma / Blurring**

**2.3.1 Blur yöntemi seçimi: Median Blur**

In [34]:
import cv2
import numpy as np

def denoise_blur(img_gray_float, ksize=5, method="median"):
    """
    img_gray_float: [0,1] aralığında float grayscale
    return: [0,1] aralığında float grayscale
    """
    img_u8 = (np.clip(img_gray_float, 0, 1) * 255).astype(np.uint8)

    if method == "median":
        # ksize tek sayı olmalı (3,5,7,...)
        out_u8 = cv2.medianBlur(img_u8, ksize)
    elif method == "gaussian":
        out_u8 = cv2.GaussianBlur(img_u8, (ksize, ksize), 0)
    else:
        raise ValueError("method 'median' veya 'gaussian' olmalı")

    return out_u8.astype(np.float32) / 255.0


**2.3.2 9 örnek: dönüşüm öncesi/sonrası görseller**

In [35]:
import random
import matplotlib.pyplot as plt

sample_paths = random.sample(image_paths, 9)

plt.figure(figsize=(12, 18))

for i, p in enumerate(sample_paths):
    print(f"Processing image: {p}") # Added for debugging
    rgb = imread(p)

    # 1) Artefakt bastırma (çıktı uint8 grayscale)
    gray_clean_u8 = hair_removal_rgb(rgb)              # uint8 [0,255]
    gray_clean = gray_clean_u8.astype(np.float32) / 255.0  # float [0,1]

    # 2) Crop
    cropped = center_crop(gray_clean, crop_ratio=0.1)

    # 3) Kontrast (Histogram Equalization)
    enhanced = contrast_enhancement(cropped)           # skimage equalize_hist -> float [0,1]

    # 4) Blur (Median)
    blurred = denoise_blur(enhanced, ksize=5, method="median")

    # Plot
    plt.subplot(9, 2, 2*i+1)
    plt.imshow(enhanced, cmap="gray")
    plt.title("Kontrast Sonrası (Blur Öncesi)")
    plt.axis("off")

    plt.subplot(9, 2, 2*i+2)
    plt.imshow(blurred, cmap="gray")
    plt.title("Median Blur (Sonrası)")
    plt.axis("off")

plt.tight_layout()
plt.show()

Processing image: /content/drive/MyDrive/ISIC_data/extracted/Skin cancer ISIC The International Skin Imaging Collaboration/Train/pigmented benign keratosis/ISIC_0025466.jpg
Processing image: /content/drive/MyDrive/ISIC_data/extracted/Skin cancer ISIC The International Skin Imaging Collaboration/Train/nevus/ISIC_0000094.jpg
Processing image: /content/drive/MyDrive/ISIC_data/extracted/Skin cancer ISIC The International Skin Imaging Collaboration/Train/vascular lesion/ISIC_0032919.jpg
Processing image: /content/drive/MyDrive/ISIC_data/extracted/Skin cancer ISIC The International Skin Imaging Collaboration/Train/basal cell carcinoma/ISIC_0026192.jpg
Processing image: /content/drive/MyDrive/ISIC_data/extracted/Skin cancer ISIC The International Skin Imaging Collaboration/Train/basal cell carcinoma/ISIC_0031095.jpg
Processing image: /content/drive/MyDrive/ISIC_data/extracted/Skin cancer ISIC The International Skin Imaging Collaboration/Train/basal cell carcinoma/ISIC_0027629.jpg
Processing i

**3 - Thresholding ile Segmentasyon**



**3.0 Yardımcı: pipeline çıktısını üret (blurred görüntü)**

In [36]:
from skimage import exposure

def preprocess_to_blur(img_rgb):
    # 1) Kıl bastırma (uint8 gray)
    gray_u8 = hair_removal_rgb(img_rgb)
    gray = gray_u8.astype(np.float32) / 255.0

    # 2) Crop
    cropped = center_crop(gray, crop_ratio=0.1)

    # 3) Kontrast (Histogram Equalization)
    enhanced = exposure.equalize_hist(cropped).astype(np.float32)

    # 4) Blur (Median)
    blurred = denoise_blur(enhanced, ksize=5, method="median")
    return blurred


**3.1 Threshold değerlerini hesapla (Global + Otsu + Try-all’den 1 yöntem)**

In [37]:
import numpy as np
import pandas as pd
from skimage.filters import threshold_otsu, threshold_yen, try_all_threshold

def compute_thresholds(img_gray_float):
    """
    img_gray_float: [0,1] float
    """
    # Global threshold: sabit bir değer seçiyoruz (genelde 0.5 başlangıçtır)
    # İstersen bunu histogramına bakıp 0.45/0.55 gibi ayarlayabiliriz.
    t_global = 0.5

    # Otsu: otomatik eşik
    t_otsu = float(threshold_otsu(img_gray_float))

    # try_all_threshold içinden bir yöntem seçiyoruz: örnek olarak Yen (otomatik)
    # (try_all_threshold görselleştirme içindir; burada seçtiğimiz yöntemin eşik değerini raporluyoruz)
    t_yen = float(threshold_yen(img_gray_float))

    return {
        "Global": t_global,
        "Otsu": t_otsu,
        "Yen (try_all seçimi)": t_yen
    }


**3.1 Çıktı: 9 örnek için eşik değerlerini print + tablo**

In [38]:
import random
from skimage.io import imread

sample_paths = random.sample(image_paths, 9)

rows = []
for p in sample_paths:
    rgb = imread(p)
    blurred = preprocess_to_blur(rgb)
    ths = compute_thresholds(blurred)

    rows.append({
        "image_path": p,
        "Global_T": ths["Global"],
        "Otsu_T": ths["Otsu"],
        "Yen_T": ths["Yen (try_all seçimi)"]
    })

thr_df = pd.DataFrame(rows)

print("Eşik değerleri (9 örnek):")
print(thr_df[["Global_T", "Otsu_T", "Yen_T"]])

thr_df


Eşik değerleri (9 örnek):
   Global_T    Otsu_T     Yen_T
0       0.5  0.480530  0.620604
1       0.5  0.480530  0.142019
2       0.5  0.472748  0.313220
3       0.5  0.453294  0.180928
4       0.5  0.449403  0.130346
5       0.5  0.499985  0.484421
6       0.5  0.492203  0.745113
7       0.5  0.472748  0.149801
8       0.5  0.499985  0.258747


3.2 Binary maskeleri üret + 9 örnekte karşılaştırmalı görselleştir

In [39]:
import matplotlib.pyplot as plt

def apply_threshold(img_gray_float, t):
    # Lezyon genelde daha koyu olabilir; hangi tarafın lezyon olduğuna göre tersine çevirebiliriz.
    # Başlangıç olarak: img < t -> lezyon (1)
    return (img_gray_float < t).astype(np.uint8)

plt.figure(figsize=(16, 24))

for i, p in enumerate(sample_paths):
    rgb = imread(p)
    blurred = preprocess_to_blur(rgb)
    ths = compute_thresholds(blurred)

    mask_global = apply_threshold(blurred, ths["Global"])
    mask_otsu   = apply_threshold(blurred, ths["Otsu"])
    mask_yen    = apply_threshold(blurred, ths["Yen (try_all seçimi)"])

    # 4 sütun: giriş + 3 maske
    plt.subplot(9, 4, 4*i+1)
    plt.imshow(blurred, cmap="gray")
    plt.title("Preprocessed (Blur Sonrası)")
    plt.axis("off")

    plt.subplot(9, 4, 4*i+2)
    plt.imshow(mask_global, cmap="gray")
    plt.title(f"Global (T={ths['Global']:.3f})")
    plt.axis("off")

    plt.subplot(9, 4, 4*i+3)
    plt.imshow(mask_otsu, cmap="gray")
    plt.title(f"Otsu (T={ths['Otsu']:.3f})")
    plt.axis("off")

    plt.subplot(9, 4, 4*i+4)
    plt.imshow(mask_yen, cmap="gray")
    plt.title(f"Yen (T={ths['Yen (try_all seçimi)']:.3f})")
    plt.axis("off")

plt.tight_layout()
plt.show()


**try_all_threshold’i 1 örnekte gösterme**

In [40]:
from skimage.filters import try_all_threshold

rgb = imread(sample_paths[0])
blurred = preprocess_to_blur(rgb)

fig, ax = try_all_threshold(blurred, figsize=(10, 8), verbose=False)
plt.show()


**4 - Post-Processing**

**4.1 Morfolojik Operatörler**

In [43]:
def morph_postprocess_safe(mask_bin, ksize_close=5):
    """
    Güvenli post-processing:
    - SADECE closing
    - küçük kernel
    - ROI tamamen silinirse geri al
    """
    m = (mask_bin * 255).astype(np.uint8)

    kernel = cv2.getStructuringElement(
        cv2.MORPH_ELLIPSE, (ksize_close, ksize_close)
    )

    closed = cv2.morphologyEx(m, cv2.MORPH_CLOSE, kernel)

    # Güvenlik: eğer ROI yok olduysa orijinale dön
    if np.sum(closed) == 0:
        closed = m.copy()

    return (closed > 0).astype(np.uint8)


**9 örnek: giriş maske + morfoloji sonrası**

In [44]:
plt.figure(figsize=(12, 18))

for i, p in enumerate(sample_paths):
    rgb = imread(p)
    _, t, mask_otsu = otsu_mask_from_rgb(rgb)

    mask_morph = morph_postprocess_safe(mask_otsu, ksize_close=5)

    plt.subplot(9, 2, 2*i+1)
    plt.imshow(mask_otsu, cmap="gray")
    plt.title(f"Giriş Maske (Otsu T={t:.3f})")
    plt.axis("off")

    plt.subplot(9, 2, 2*i+2)
    plt.imshow(mask_morph, cmap="gray")
    plt.title("Morfoloji Sonrası (Closing)")
    plt.axis("off")

plt.tight_layout()
plt.show()


**4.2 Connected Component Labeling (CCL)**

**4.2.0 — Yardımcı fonksiyonlar**

In [49]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
from collections import Counter

def connected_components(mask_bin):
    """
    mask_bin: 0/1 uint8
    return: num_labels (bg dahil), labels, fg_count (bg hariç)
    """
    num_labels, labels = cv2.connectedComponents(mask_bin.astype(np.uint8))
    fg_count = num_labels - 1
    return num_labels, labels, fg_count

def labels_to_color(labels):
    """CCL label haritasını renkli görselleştirir."""
    if labels.max() == 0:
        return np.zeros((*labels.shape, 3), dtype=np.uint8)
    lab_u8 = (labels.astype(np.float32) / labels.max() * 255).astype(np.uint8)
    colored = cv2.applyColorMap(lab_u8, cv2.COLORMAP_JET)
    colored = cv2.cvtColor(colored, cv2.COLOR_BGR2RGB)
    return colored

def remove_small_components(mask_bin, min_area=500):
    """Küçük bağlı bileşenleri (gürültü) eler."""
    num, lab = cv2.connectedComponents(mask_bin.astype(np.uint8))
    out = np.zeros_like(mask_bin, dtype=np.uint8)
    for k in range(1, num):
        area = int(np.sum(lab == k))
        if area >= min_area:
            out[lab == k] = 1
    return out

def keep_largest_component(mask_bin):
    """En büyük alanlı bağlı bileşeni seçer."""
    num, lab = cv2.connectedComponents(mask_bin.astype(np.uint8))
    if num <= 2:  # bg + 1 fg
        return mask_bin

    areas = []
    for k in range(1, num):
        areas.append((k, int(np.sum(lab == k))))
    largest = max(areas, key=lambda x: x[1])[0]
    return (lab == largest).astype(np.uint8)

def pick_single_roi(mask_bin, min_area=500):
    """
    1 görüntü = 1 ROI stratejisi:
    1) küçük bileşenleri ele (alan filtresi)
    2) en büyük bileşeni ROI olarak seç
    3) ROI çok küçük kalırsa maskeyi invert edip tekrar dene (lezyon/arka plan tersliği için)
    """
    cleaned = remove_small_components(mask_bin, min_area=min_area)
    roi = keep_largest_component(cleaned)

    # güvenlik: ROI çok küçükse tersini dene
    if np.sum(roi) < 1000:
        inv = 1 - mask_bin
        cleaned2 = remove_small_components(inv, min_area=min_area)
        roi2 = keep_largest_component(cleaned2)
        if np.sum(roi2) > np.sum(roi):
            roi = roi2

    return roi


**4.2.1 — 9 örnek: bileşen sayısını çizdir + renkli label plot**

In [51]:
sample_paths = random.sample(image_paths, 9)

component_counts_plot = []

plt.figure(figsize=(14, 18))

for i, p in enumerate(sample_paths):
    rgb = imread(p)

    # 4.1 çıktısı
    _, _, mask_otsu = otsu_mask_from_rgb(rgb)
    mask_morph = morph_postprocess_safe(mask_otsu, ksize_close=5)

    # CCL
    _, labels_before, fg_before = connected_components(mask_morph)
    component_counts_plot.append(fg_before)

    # Renkli label
    colored = labels_to_color(labels_before)

    plt.subplot(9, 2, 2*i+1)
    plt.imshow(colored)
    plt.title(f"CCL Label (fg={fg_before})")
    plt.axis("off")

    plt.subplot(9, 2, 2*i+2)
    plt.bar(["Bileşen Sayısı"], [fg_before])
    plt.ylim(0, max(component_counts_plot) + 5)
    plt.title("Bileşen Sayısı (bg hariç)")
    plt.grid(axis="y", alpha=0.3)

plt.tight_layout()
plt.show()
plt.close()


**5 - Öznitelik (Feature) Çıkarımı**

**5.1 First-Order (İstatistiksel) Özellikler**

In [52]:
import numpy as np
from scipy.stats import skew, kurtosis

def first_order_features(gray_img, roi_mask):
    pixels = gray_img[roi_mask == 1]

    feats = {}
    feats["mean"] = float(np.mean(pixels))
    feats["std"] = float(np.std(pixels))
    feats["variance"] = float(np.var(pixels))
    feats["min"] = float(np.min(pixels))
    feats["max"] = float(np.max(pixels))
    feats["median"] = float(np.median(pixels))
    feats["skewness"] = float(skew(pixels))
    feats["kurtosis"] = float(kurtosis(pixels))

    # Entropy
    hist, _ = np.histogram(pixels, bins=256, density=True)
    hist = hist[hist > 0]
    feats["entropy"] = float(-np.sum(hist * np.log2(hist)))

    # Energy
    feats["energy"] = float(np.sum(pixels ** 2))

    return feats


**5.2 2D Shape (Şekil) Özellikleri**

In [53]:
from skimage.measure import regionprops, label

def shape_features(roi_mask):
    lbl = label(roi_mask)
    props = regionprops(lbl)[0]   # 1 ROI varsayımı

    feats = {}
    feats["area"] = props.area
    feats["perimeter"] = props.perimeter
    feats["eccentricity"] = props.eccentricity
    feats["solidity"] = props.solidity
    feats["extent"] = props.extent

    feats["major_axis_length"] = props.major_axis_length
    feats["minor_axis_length"] = props.minor_axis_length
    feats["aspect_ratio"] = props.major_axis_length / props.minor_axis_length

    feats["convex_area"] = props.convex_area
    feats["equivalent_diameter"] = props.equivalent_diameter

    # Circularity
    feats["circularity"] = (4 * np.pi * props.area) / (props.perimeter ** 2)

    # Bounding box
    minr, minc, maxr, maxc = props.bbox
    feats["bbox_width"] = maxc - minc
    feats["bbox_height"] = maxr - minr

    return feats


**5.3 GLCM (Second-Order Texture) Özellikleri**

In [54]:
from skimage.feature import graycomatrix, graycoprops

def glcm_features(gray_img, roi_mask,
                  distances=[1, 2],
                  angles=[0, np.pi/4, np.pi/2, 3*np.pi/4],
                  levels=16):

    # ROI dışını maskele
    masked = gray_img.copy()
    masked[roi_mask == 0] = 0

    # Quantization
    img_q = np.floor(masked * (levels - 1)).astype(np.uint8)

    glcm = graycomatrix(
        img_q,
        distances=distances,
        angles=angles,
        levels=levels,
        symmetric=True,
        normed=True
    )

    feats = {}
    for prop in ["contrast", "dissimilarity", "homogeneity",
                 "energy", "correlation", "ASM"]:
        feats[f"glcm_{prop}"] = float(np.mean(graycoprops(glcm, prop)))

    return feats


**5.4 Feature Tablosu (CSV)**

In [70]:

import numpy as np
import pandas as pd
import cv2

from tqdm.notebook import tqdm
from skimage.io import imread
from skimage.feature import graycomatrix, graycoprops
from skimage.measure import label, regionprops


MIN_AREA = 500
GLCM_LEVELS = 8
GLCM_DISTANCES = [1]
GLCM_ANGLES = [0, np.pi/2]
GLCM_PATCH_SIZE = 128


def safe_imread(path):
    try:
        x = imread(path)
        return x if x is not None else None
    except:
        return None


def crop_to_roi_patch(gray_img, roi_mask, patch_size=GLCM_PATCH_SIZE):
    props = regionprops(label(roi_mask))
    if len(props) == 0:
        return None, None

    cy, cx = map(int, props[0].centroid)
    half = patch_size // 2

    y1 = max(cy - half, 0)
    y2 = min(cy + half, gray_img.shape[0])
    x1 = max(cx - half, 0)
    x2 = min(cx + half, gray_img.shape[1])

    g = gray_img[y1:y2, x1:x2]
    m = roi_mask[y1:y2, x1:x2].astype(np.uint8)

    # patch boyutu tutmazsa padding
    if g.shape[0] != patch_size or g.shape[1] != patch_size:
        g_pad = np.zeros((patch_size, patch_size), dtype=g.dtype)
        m_pad = np.zeros((patch_size, patch_size), dtype=m.dtype)
        g_pad[:g.shape[0], :g.shape[1]] = g
        m_pad[:m.shape[0], :m.shape[1]] = m
        g, m = g_pad, m_pad

    return g, m


def glcm_feats(gray_img, roi_mask):
    g, m = crop_to_roi_patch(gray_img, roi_mask, patch_size=GLCM_PATCH_SIZE)
    if g is None:
        return {f"glcm_{p}": np.nan for p in ["contrast","dissimilarity","homogeneity","energy","correlation","ASM"]}

    masked = g.copy()
    masked[m == 0] = 0

    # gray [0,1] değilse normalize
    if masked.max() > 1.0:
        masked = masked / 255.0

    img_q = np.floor(np.clip(masked, 0, 1) * (GLCM_LEVELS - 1)).astype(np.uint8)

    glcm = graycomatrix(
        img_q,
        distances=GLCM_DISTANCES,
        angles=GLCM_ANGLES,
        levels=GLCM_LEVELS,
        symmetric=True,
        normed=True
    )

    out = {}
    for p in ["contrast","dissimilarity","homogeneity","energy","correlation","ASM"]:
        out[f"glcm_{p}"] = float(np.mean(graycoprops(glcm, p)))
    return out


rows = []
skipped = 0

print("✅ GLCM Parametreleri:",
      "levels=", GLCM_LEVELS,
      "distances=", GLCM_DISTANCES,
      "angles=", GLCM_ANGLES,
      "patch=", GLCM_PATCH_SIZE)

for p in tqdm(image_paths, desc="Feature extraction", unit="image"):
    rgb = safe_imread(p)
    if rgb is None:
        skipped += 1
        continue

    image_id = p.split("/")[-1]

    gray = preprocess_to_blur(rgb)

    _, _, mask_otsu = otsu_mask_from_rgb(rgb)
    mask_morph = morph_postprocess_safe(mask_otsu, ksize_close=5)
    mask_final = pick_single_roi(mask_morph, min_area=MIN_AREA)

    row = {"image_id": image_id, "class": "unknown"}
    row.update(first_order_features(gray, mask_final))
    row.update(shape_features(mask_final))
    row.update(glcm_feats(gray, mask_final))

    rows.append(row)

df = pd.DataFrame(rows)
df.to_csv("lesion_features.csv", index=False)

print("✅ Bitti.")
print("Toplam görüntü (liste):", len(image_paths))
print("İşlenen satır (ROI) sayısı:", df.shape[0])
print("Feature sütun sayısı:", df.shape[1])
print("Atlanan (okunamayan) dosya:", skipped)
print("CSV:", "lesion_features.csv")

df.head()


✅ GLCM Parametreleri: levels= 8 distances= [1] angles= [0, 1.5707963267948966] patch= 128


✅ Bitti.
Toplam görüntü (liste): 2357
İşlenen satır (ROI) sayısı: 2356
Feature sütun sayısı: 31
Atlanan (okunamayan) dosya: 1
CSV: lesion_features.csv


<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

